# Лабораторная работа №2
# Глобальная оптимизация и метаэврестические алгоритмы
# Задание на лабораторную работу:
- В Pygmo запрогроммировать две своих тестовых функции и найти их оптимум 3 разными алгоритмами доступными в библиотеке и получить таблицу сравнения

In [14]:
!pip3 install pygmo

In [22]:
import math

import numpy as np
import pygmo as pg
import plotly.graph_objects as go
import pandas as pd

In [16]:
class TestFunction():
  def __init__(self):
    self.title = 'Test Function'
    self.minimum = np.array([0, 0, 0])
    self.X_idx = 0
    self.Y_idx = 1
    self.left_def = 0
    self.right_def = 1


  def visualize_func(self):
    """
    Визуализbhetn функцию в графике plotly
    """
    x_definability, y_definability = self.definability_of_func()[self.X_idx], self.definability_of_func()[self.Y_idx]
    x_values = np.arange(x_definability[self.left_def], x_definability[self.right_def])
    y_values = np.arange(y_definability[self.left_def], y_definability[self.right_def])

    z_values = []
    for xs in x_values:
      values = []
      for ys in y_values:
        values.append(self.function(xs,ys))
      z_values.append(values)

    fig = go.Figure(data=[go.Surface(z=z_values, x=x_values, y=y_values)])
    fig.update_layout(title=self.title)
    fig.show()

  def definability_of_func(self):
    """
    Метод выдает 2 параметра: левая и правая граница определения функции.
    Если присутствует x и y, то будет массив, где будет сначала x, потом y.
    """
    pass

  def function(self, x, y):
    """
    Определение самой функции. Выдается значение для 3-х мерной функции.
    То есть выдается Z
    """
    pass

  def fitness(self, dv):
    assert len(dv) == 2
    x = dv[0]
    y = dv[1]
    return [self.function(x,y)]

  def get_bounds(self):
    x_definability, y_definability = np.array(self.definability_of_func()[self.X_idx]), np.array(self.definability_of_func()[self.Y_idx])
    return ([x_definability[self.left_def], y_definability[self.left_def]], [x_definability[self.right_def], y_definability[self.right_def]])

In [17]:
class BoothFunction(TestFunction):
  def __init__(self):
      super().__init__()
      self.title = "Booth's Function"
      self.minimum = np.array([1, 1, 0])

  def definability_of_func(self):
    return ([-10, 10], [-10, 10])

  def function(self, x, y):
    return (x + 2 * y - 7) ** 2 + (2 * x + y - 5) ** 2

class LevyFunction(TestFunction):
  def __init__(self):
      super().__init__()
      self.title = "Levy Function N. 13"
      self.minimum = np.array([1, 1, 0])

  def definability_of_func(self):
    return ([-10, 10], [-10, 10])

  def function(self, x, y):
    return (np.sin(3 * np.pi * x) ** 2 +
                (x - 1) ** 2 * (1 + np.sin(3 * np.pi * y) ** 2) +
                (y - 1) ** 2 * (1 + np.sin(2 * np.pi * x) ** 2))

In [29]:
booth_func = BoothFunction()
booth_func.visualize_func()

In [40]:
levy_func = LevyFunction()
levy_func.visualize_func()


Были выбраны следующие методы оптимизации:

- Bee Colony
- Grey wolf optimizer
- Differential Evolution

In [115]:
def apply_algorithms(func: TestFunction,
                     df,
                     algorithms=[pg.bee_colony(gen = 100),  pg.gwo(gen = 100), pg.de(gen = 100)]):
  best_solutions = []
  for algo in algorithms:
    pr = pg.problem(func)
    algo = pg.algorithm(algo)
    population = pg.population(pr, size = 100)
    population = algo.evolve(population)
    best_fitness = population.champion_x
    best_solution = population.champion_f
    best_solutions.append(f"{best_fitness} {best_solution}")
  df.loc[len(df.index)] = [func.title] + best_solutions
  return df

In [116]:
df = pd.DataFrame({'Algorithms':[], 'Bee Colony':[], 'Grey wolf optimizer':[], 'Differential Evolution':[]})
df = apply_algorithms(booth_func, df)
df = apply_algorithms(levy_func, df)
df

,Algorithms,Bee Colony,Grey wolf optimizer,Differential Evolution
0,Booth's Function,[0.9995936 3.00075845] [1.23618199e-06],[1.00033607 3.00001315] [6.00931582e-07],[1.00004631 2.99996875] [4.02781612e-09]
1,Levy Function N. 13,[1. 1.] [1.23876479e-23],[0.99999125 1.00007641] [1.2717002e-08],[0.99999651 0.99999254] [1.15050083e-09]
